In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

predictingFY = 18
FY_end = dt.strptime('05/31/2017', '%m/%d/%Y')
pd.set_option('display.max_columns', None) 

In [ ]:
data = pd.read_csv('./report.csv', dtype='str')
data.columns = data.columns.str.replace(' ', '_')
data.columns = data.columns.str.replace(':', '')
data.columns = data.columns.str.replace('.', '')

In [ ]:
data.head()

In [ ]:
# create map to store contact giving information
con_giving = {}
for contactId in data['Contact_ID'] :
    con_giving[contactId] = {'Largest_Gift':0, 'Num_Gifts_LFY':0, 'Num_Gifts_2FY':0, 'Num_Gifts_3FY':0, 
                             'Num_Gifts':0, 'FY_Given':set(), 'SC_Giving': 0, 'HC_Giving':0, 'Balance_Expected_NFY':0}

In [ ]:
# go through each row to summarize contact information
for row in data.itertuples():
    if not pd.isnull(row.Fiscal_Year):
        rowFY = int(row.Fiscal_Year[-2:])
        if rowFY != predictingFY and rowFY != predictingFY + 1 :
            if (pd.isnull(row.Stage) or row.Stage == 'Completed') and row.Record_Type not in ('Split Gift', 'Pledge', 'Intention', 'Pledge Canceled Value'):  
                con_giving[row.Contact_ID]['FY_Given'].add(rowFY)
                con_giving[row.Contact_ID]['Num_Gifts'] += 1

                if con_giving[row.Contact_ID]['Largest_Gift'] < float(row.Analytics_Amount):
                    con_giving[row.Contact_ID]['Largest_Gift'] = float(row.Analytics_Amount)

                if rowFY == (predictingFY - 1):  
                    con_giving[row.Contact_ID]['Num_Gifts_LFY'] += 1
                elif rowFY == (predictingFY - 2):  
                    con_giving[row.Contact_ID]['Num_Gifts_2FY'] += 1
                elif rowFY == (predictingFY - 3):  
                    con_giving[row.Contact_ID]['Num_Gifts_3FY'] += 1

                if row.Record_Type == 'Credit':
                    con_giving[row.Contact_ID]['SC_Giving'] += float(row.Analytics_Amount)
                else:
                    con_giving[row.Contact_ID]['HC_Giving'] += float(row.Analytics_Amount)
        elif rowFY == predictingFY: 
            if row.Record_Type in ('Payment', 'Intention'):
                con_giving[row.Contact_ID]['Balance_Expected_NFY'] += float(row.Analytics_Amount)

In [ ]:
# now that I'm done using giving data, we can get rid of all duplicate contact Id rows and giving info
data = data.drop_duplicates(subset='Contact_ID', keep='first')
data = data.drop(['Analytics_Amount', 'Campaign_Code', 'Proposal_Opportunity_Name', 
                  'Record_Type', 'Stage', 'Type', 'Fiscal_Year', 'Giving_ID'], axis=1)
data = data.set_index('Contact_ID')

In [ ]:
data['Largest_Gift'] = 0
data['Num_Gifts_LFY'] = 0
data['Num_Gifts_2FY'] = 0
data['Num_Gifts_3FY'] = 0
data['Num_Gifts'] = 0
data['Num_FY_Given'] = 0
data['HC_Giving'] = 0
data['SC_Giving'] = 0
data['Balance_Expected_NFY'] = 0
data['SC_Giving'] = 0
data['Avg_Gift_Amount'] = 0
data['Avg_Gifts_Per_Year'] = 0
data['All_Giving'] = 0
data['num_active_comm_groups'] = 0

In [ ]:
counter = len(con_giving)
for con_id in con_giving:
    if con_giving[con_id]['Num_Gifts'] > 0:
        print(counter)
        data.loc[con_id, 'Largest_Gift'] = con_giving[con_id]['Largest_Gift']
        data.loc[con_id, 'Num_Gifts_LFY'] = con_giving[con_id]['Num_Gifts_LFY']
        data.loc[con_id, 'Num_Gifts_2FY'] = con_giving[con_id]['Num_Gifts_2FY']
        data.loc[con_id, 'Num_Gifts_3FY'] = con_giving[con_id]['Num_Gifts_3FY']
        data.loc[con_id, 'Num_Gifts'] = con_giving[con_id]['Num_Gifts']
        data.loc[con_id, 'Num_FY_Given'] = len(con_giving[con_id]['FY_Given'])
        data.loc[con_id, 'Num_Gifts'] = con_giving[con_id]['Num_Gifts']
        data.loc[con_id, 'HC_Giving'] = con_giving[con_id]['HC_Giving']
        data.loc[con_id, 'SC_Giving'] = con_giving[con_id]['SC_Giving']
        data.loc[con_id, 'Balance_Expected_NFY'] = con_giving[con_id]['Balance_Expected_NFY']
        data.loc[con_id, 'SC_Giving'] = con_giving[con_id]['SC_Giving']
        all_giving = float(con_giving[con_id]['HC_Giving']) + float(con_giving[con_id]['SC_Giving'])
        data.loc[con_id, 'All_Giving'] = all_giving
        data.loc[con_id, 'Avg_Gift_Amount'] = all_giving/con_giving[con_id]['Num_Gifts']
        data.loc[con_id, 'Avg_Gifts_Per_Year'] = con_giving[con_id]['Num_Gifts']/len(con_giving[con_id]['FY_Given'])
        counter -= 1

In [ ]:
# Assign value 0-3 based on prospect priority
data['prospect_priority'] = 0
data.loc[data.Prospect_Priority == 'top', 'prospect_priority'] = 3 
data.loc[data.Prospect_Priority == 'mid', 'prospect_priority'] = 2 
data.loc[data.Prospect_Priority == 'low', 'prospect_priority'] = 1 

data = data.drop(['Prospect_Priority'], axis=1)

In [ ]:
# Assign value based on top constituency for account
data['acct_top_const'] = 0
data.loc[data.Top_Priority_Constituency.isin(['Alumni-Graduate', 'Non-Person Portfolio', 'Plaque', 'Student Ministries Only Donor']), 'acct_top_const'] = 15
data.loc[data.Top_Priority_Constituency.isin(['Alumni-Nongraduate', 'Student']), 'acct_top_const'] = 14
data.loc[data.Top_Priority_Constituency.isin(['Current Parent']), 'acct_top_const'] = 13
data.loc[data.Top_Priority_Constituency.isin(['Parent']), 'acct_top_const'] = 12
data.loc[data.Top_Priority_Constituency.isin(['Grandparent']), 'acct_top_const'] = 11
data.loc[data.Top_Priority_Constituency.isin(['Faculty', 'Former Emp Unkown']), 'acct_top_const'] = 10
data.loc[data.Top_Priority_Constituency.isin(['Alumni-Nongraduate', 'Student', 'Former Faculty', 'Former Staff', 'Staff', 'Professor']), 'acct_top_const'] = 9
data.loc[data.Top_Priority_Constituency.isin(['Professor Emeritus']), 'acct_top_const'] = 8
data.loc[data.Top_Priority_Constituency.isin(['Friend']), 'acct_top_const'] = 7
data.loc[data.Top_Priority_Constituency.isin(['Fifth Year Program']), 'acct_top_const'] = 6
data.loc[data.Top_Priority_Constituency.isin(['Trustee', 'Trustee Emeritus', 'Former Trustee']), 'acct_top_const'] = 5
data.loc[data.Top_Priority_Constituency.isin(['Special Mailing List', 'Women\'s Leadership Council']), 'acct_top_const'] = 4
data.loc[data.Top_Priority_Constituency.isin(['Museum Mailing List', 'Potential Student', 'Women\'s Leadership Tea', 'Host Parent']), 'acct_top_const'] = 3
data.loc[data.Top_Priority_Constituency.isin(['Trailhead']), 'acct_top_const'] = 2
data.loc[data.Top_Priority_Constituency.isin(['Consortium Visitor', 'Possible Duplicate']), 'acct_top_const'] = 1

data = data.drop(['Top_Priority_Constituency'], axis=1)

In [ ]:
# Assign value based on top constituency for contact
data['con_top_const'] = 0
data.loc[data.Top_Priority_Constituency1.isin(['Alumni-Graduate', 'Non-Person Portfolio', 'Plaque', 'Student Ministries Only Donor']), 'con_top_const'] = 15
data.loc[data.Top_Priority_Constituency1.isin(['Alumni-Nongraduate', 'Student']), 'con_top_const'] = 14
data.loc[data.Top_Priority_Constituency1.isin(['Current Parent']), 'con_top_const'] = 13
data.loc[data.Top_Priority_Constituency1.isin(['Parent']), 'con_top_const'] = 12
data.loc[data.Top_Priority_Constituency1.isin(['Grandparent']), 'con_top_const'] = 11
data.loc[data.Top_Priority_Constituency1.isin(['Faculty', 'Former Emp Unkown']), 'con_top_const'] = 10
data.loc[data.Top_Priority_Constituency1.isin(['Alumni-Nongraduate', 'Student', 'Former Faculty', 'Former Staff', 'Staff', 'Professor']), 'con_top_const'] = 9
data.loc[data.Top_Priority_Constituency1.isin(['Professor Emeritus']), 'con_top_const'] = 8
data.loc[data.Top_Priority_Constituency1.isin(['Friend']), 'con_top_const'] = 7
data.loc[data.Top_Priority_Constituency1.isin(['Fifth Year Program']), 'con_top_const'] = 6
data.loc[data.Top_Priority_Constituency1.isin(['Trustee', 'Trustee Emeritus', 'Former Trustee']), 'con_top_const'] = 5
data.loc[data.Top_Priority_Constituency1.isin(['Special Mailing List', 'Women\'s Leadership Council']), 'con_top_const'] = 4
data.loc[data.Top_Priority_Constituency1.isin(['Museum Mailing List', 'Potential Student', 'Women\'s Leadership Tea', 'Host Parent']), 'con_top_const'] = 3
data.loc[data.Top_Priority_Constituency1.isin(['Trailhead']), 'acct_top_const'] = 2
data.loc[data.Top_Priority_Constituency1.isin(['Consortium Visitor', 'Possible Duplicate']), 'con_top_const'] = 1

data = data.drop(['Top_Priority_Constituency1'], axis=1)

In [ ]:
# Assign value based on account stage
data['stage'] = 0
data.loc[data.Stage1.isin(['Parking Lot', 'Blank Stage']), 'stage'] = -1
data.loc[data.Stage1.isin(['Prospecting', 'Auxiliary Portfolio', 'Volunteer Networker']), 'stage'] = 1
data.loc[data.Stage1.isin(['Discovery']), 'stage'] = 2
data.loc[data.Stage1 == 'Cultivation', 'stage'] = 3
data.loc[data.Stage1 == 'Solicitation', 'stage'] = 4
data.loc[data.Stage1 == 'Stewardship', 'stage'] = 5

data = data.drop(['Stage1'], axis=1)

In [ ]:
# Make home value null if they couldn't find or calculate
data.loc[data.Home_Value == str(0), 'Home_Value'] = np.nan
data.loc[data.Home_Value_Missing == 'Non SFR', 'Home_Value'] = 0
data = data.drop(['Home_Value_Missing'], axis=1)

In [ ]:
# Make capacity low value null if 0
data.loc[data.Capacity_Low_Value == str(0), 'Capacity_Low_Value'] = np.nan

In [ ]:
# Make age null if 0
data.loc[data.Age == str(0), 'Age'] = np.nan

In [ ]:
# calculate gender mean, capacity low value, and age mean to fill in null values
ageSum = 0
ageCount = 0
genderSum = 0
genderCount = 0
capacitySum = 0
capacityCount = 0
homeSum = 0
homeCount = 0

for row in data.itertuples():
    # add number of active communication groups
    split = str(row.Active_Communication_Groups).split('\n')
    data.at[row.Index, 'num_active_comm_groups'] = len(split)
    
    if not pd.isnull(row.Age):
        ageSum += int(row.Age)
        ageCount += 1
    if not pd.isnull(row.Gender):
        genderCount += 1
        if row.Gender == 'Female':
            genderSum += 1
    if not pd.isnull(row.Capacity_Low_Value):
        capacitySum += int(row.Capacity_Low_Value)
        capacityCount += 1
    if not pd.isnull(row.Home_Value):
        homeSum += int(row.Home_Value)
        homeCount += 1
        
data = data.drop(['Active_Communication_Groups'], axis=1) 

# Assign 0 if male, 1 if female, mean if null
data['gender'] = int(genderSum/genderCount)

data.loc[data.Gender == 'Male', 'gender'] = 0 
data.loc[data.Gender == 'Female', 'gender'] = 1 

data = data.drop(['Gender'], axis=1)

# Assign mean age to null ages
data.loc[pd.isnull(data.Age), 'Age'] = int(ageSum/ageCount)

# Assign mean capacity value to null cells
data.loc[pd.isnull(data.Capacity_Low_Value), 'Capacity_Low_Value'] = int(capacitySum/capacityCount)

# Assign mean home value to null cells
data.loc[pd.isnull(data.Home_Value), 'Home_Value'] = int(homeSum/homeCount)

In [ ]:
# if contact has had a contact report 1, else 0
data['had_contact_report'] = np.where(data['First_Contact_Report_Date'].isnull(), 0, 1)
data = data.drop(['First_Contact_Report_Date'], axis=1)

In [ ]:
# if in portfolio, 1, else 0
data['portfolio_1'] = np.where(data['Primary_Portfolio_Employee_Name'].isnull(), 0, 1)
data = data.drop(['Primary_Portfolio_Employee_Name'], axis=1)

data['portfolio_2'] = np.where(data['Secondary_Portfolio_Employee_Name'].isnull(), 0, 1)
data = data.drop(['Secondary_Portfolio_Employee_Name'], axis=1)

data['portfolio_3'] = np.where(data['Tertiary_Portfolio_Employee_Name'].isnull(), 0, 1)
data = data.drop(['Tertiary_Portfolio_Employee_Name'], axis=1)

In [ ]:
# if account is in a giving society, 1, else 0
data['in_giving_society'] = np.where(data['Giving_Societies'].isnull(), 0, 1)
data = data.drop(['Giving_Societies'], axis=1)

In [ ]:
# if contact graduated from westmont, 1, else 0
data['westmont_grad'] = np.where(data['Westmont_Education_Status'] == 'Graduated', 1, 0)
data = data.drop(['Westmont_Education_Status'], axis=1)

In [ ]:
# add event attendance
event_data = pd.read_csv('./event-report.csv', dtype='str')
event_data.columns = event_data.columns.str.replace(' ', '_')
event_data.columns = event_data.columns.str.replace(':', '')

In [ ]:
event_attendance = {}
for contactId in event_data['Contact_Contact_ID'] :
    event_attendance[contactId] = 0

In [ ]:
for row in event_data.itertuples():
    event_attendance[row.Contact_Contact_ID] += 1

In [ ]:
data['Num_Events_RSVP'] = 0
counter = len(event_attendance)
for con_id in event_attendance:
    print(counter)
    data.loc[con_id, 'Num_Events_RSVP'] = event_attendance[con_id]
    counter -= 1

In [ ]:
# add interest/activity count
ia_data = pd.read_csv('./IA-report.csv', dtype='str')
ia_data.columns = ia_data.columns.str.replace(' ', '_')
ia_data.columns = ia_data.columns.str.replace(':', '')

In [ ]:
ia = {}
for contactId in ia_data['Contact_Contact_ID'] :
    ia[contactId] = {'total':0, 'current':0}

In [ ]:
for row in ia_data.itertuples():
    if(pd.isnull(row.End_Date)):
        ia[row.Contact_Contact_ID]['total'] += 1
        ia[row.Contact_Contact_ID]['current'] += 1
        
    else:
        IA_end = dt.strptime(str(row.End_Date), '%m/%d/%Y')
        ia[row.Contact_Contact_ID]['total'] += 1
        if(IA_end < FY_end):
            ia[row.Contact_Contact_ID]['current'] += 1

In [ ]:
data['Num_Total_IA'] = 0
data['Num_Current_IA'] = 0

In [ ]:
counter = len(ia)
for con_id in ia:
    print(counter)
    data.loc[con_id, 'Num_Total_IA'] = ia[con_id]['total']
    data.loc[con_id, 'Num_Current_IA'] = ia[con_id]['current']
    counter -= 1

In [ ]:
# add board/council count
bc_data = pd.read_csv('./BC-report.csv', dtype='str')
bc_data.columns = bc_data.columns.str.replace(' ', '_')
bc_data.columns = bc_data.columns.str.replace(':', '')

In [ ]:
bc = {}
for contactId in bc_data['Contact_Contact_ID'] :
    bc[contactId] = {'total':0, 'current':0}

for row in bc_data.itertuples():
    if(pd.isnull(row.End_Date)):
        bc[row.Contact_Contact_ID]['total'] += 1
        bc[row.Contact_Contact_ID]['current'] += 1
        
    else:
        BC_end = dt.strptime(str(row.End_Date), '%m/%d/%Y')
        bc[row.Contact_Contact_ID]['total'] += 1
        if(BC_end < FY_end):
            bc[row.Contact_Contact_ID]['current'] += 1

In [ ]:
data['Num_Total_BC'] = 0
data['Num_Current_BC'] = 0

In [ ]:
counter = len(bc)
for con_id in bc:
    print(counter)
    data.loc[con_id, 'Num_Total_BC'] = bc[con_id]['total']
    data.loc[con_id, 'Num_Current_BC'] = bc[con_id]['current']
    counter -= 1

In [ ]:
data.to_csv('clean-report.csv', index=False)